In [1]:
import os
import numpy as np 
from PIL import Image
import matplotlib.pyplot as plt
from sklearn import preprocessing

Define a function to get load and preprocess the data

In [ ]:
def process_images(image_directory):
    